In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

dataset = torchvision.datasets.ImageFolder(root="dataset", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
test_dataset = torchvision.datasets.ImageFolder(root="dataset", transform=transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32)
class ImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # ... (Add more conv layers if needed)
        self.fc1 = nn.Linear(16 * 8 * 22, 120)  # Update the input size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 16 * 8 * 22)  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = ImageClassifier(num_classes=len(dataset.classes)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()



for epoch in range(10):
    model.train()
    print(f"epoch # {epoch}")
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            outputs = model(images.to(device))
            val_loss += loss_fn(outputs, labels.to(device)).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels.to(device)).sum().item()
    val_accuracy  = correct / len(test_dataset)
    print(f"Epoch {epoch+1}: Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}")



cuda
epoch # 0
Epoch 1: Validation Loss: 0.0052, Accuracy: 1.00
epoch # 1
Epoch 2: Validation Loss: 0.0028, Accuracy: 1.00
epoch # 2
Epoch 3: Validation Loss: 0.0018, Accuracy: 1.00
epoch # 3
Epoch 4: Validation Loss: 0.0012, Accuracy: 1.00
epoch # 4
Epoch 5: Validation Loss: 0.0009, Accuracy: 1.00
epoch # 5
Epoch 6: Validation Loss: 0.0007, Accuracy: 1.00
epoch # 6
Epoch 7: Validation Loss: 0.0005, Accuracy: 1.00
epoch # 7
Epoch 8: Validation Loss: 0.0004, Accuracy: 1.00
epoch # 8
Epoch 9: Validation Loss: 0.0003, Accuracy: 1.00
epoch # 9
Epoch 10: Validation Loss: 0.0003, Accuracy: 1.00


In [10]:
model.eval()
from PIL import Image
with torch.no_grad():
    img = Image.open(r"./Kiriko.png")
    img = transform(img).unsqueeze(0).to(device)
    output = model(img)

pred = torch.argmax(output, dim=1).item()
print(f"Predicted Class {dataset.classes[pred]}")

Predicted Class Kiriko
